Load the model name. The **`event_message`** is automatically populated by the webhook.

In [0]:
import json
 
event_message = dbutils.widgets.get("event_message")
event_message_dict = json.loads(event_message)
model_name = event_message_dict.get("model_name")

print(event_message_dict)
print(model_name)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2121898024981507>:3
      1 import json
----> 3 event_message = dbutils.widgets.get("event_message")
      4 event_message_dict = json.loads(event_message)
      5 model_name = event_message_dict.get("model_name")

File /databricks/python_shell/dbruntime/WidgetHandlerImpl.py:42, in WidgetsHandlerImpl.get(self, name)
     36 def get(self, name):
     37     """ Returns the current value of a widget with give name.
     38 
     39     :param name: Name of the argument to be accessed
     40     :return: Current value of the widget or default value
     41     """
---> 42     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 command = proto.CALL_COMMAND_NAME +\


Use the model name to get the latest model version.

In [0]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

version = client.get_registered_model(model_name).latest_versions[0].version
version


Use the model name and version to load a **`pyfunc`** model of our model in production.

In [0]:
import mlflow

pyfunc_model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{version}")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2121898024981511>:3
      1 import mlflow
----> 3 pyfunc_model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{version}")

NameError: name 'model_name' is not defined


Get the input and output schema of our logged model.

In [0]:
input_schema = pyfunc_model.metadata.get_input_schema().as_spark_schema()
output_schema = pyfunc_model.metadata.get_output_schema().as_spark_schema()




Here we define our expected input and output schema.

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, DoubleType

expected_input_schema = (StructType([
    StructField('fixed_acidity', DoubleType(), True), StructField('volatile_acidity', DoubleType(), True), StructField('citric_acid', DoubleType(), True), 
    StructField('residual_sugar', DoubleType(), True), StructField('chlorides', DoubleType(), True), 
    StructField('free_sulfur_dioxide', DoubleType(), True), StructField('total_sulfur_dioxide', DoubleType(), True), StructField('density', DoubleType(), True), 
    StructField('pH', DoubleType(), True), 
    StructField('sulphates', DoubleType(), True), 
    StructField('alcohol', DoubleType(), True), 
    StructField('is_red', LongType(), True)
]))

expected_output_schema = StructType([StructField('quality', LongType(), True)])

In [0]:
assert expected_input_schema.fields.sort(key=lambda x: x.name) == input_schema.fields.sort(key=lambda x: x.name)
assert expected_output_schema.fields.sort(key=lambda x: x.name) == output_schema.fields.sort(key=lambda x: x.name)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2121898024981519>:1
----> 1 StructType([
      2     StructField('quality', LongType(), True)
      3 ])

NameError: name 'StructType' is not defined


Load the dataset and generate some predictions to ensure our model is working correctly.

In [0]:
import pandas as pd

white_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-white.csv", sep=";")
red_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-red.csv", sep=";")

red_wine['is_red'] = 1
white_wine['is_red'] = 0

data = pd.concat([red_wine, white_wine], axis=0)

# Remove spaces from column names
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

high_quality = (data.quality >= 7).astype(int)
data.quality = high_quality

data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)

predictions = pyfunc_model.predict(data)

predictions


Make sure our prediction types are correct.

In [0]:
import numpy as np

assert type(predictions) == np.ndarray
assert type(predictions[0]) == np.int64